In [ ]:
log_repair_query = '''
     MATCH (e1:Event) - [e2e:DF] -> (e2:Event) - [] -> (o) - [:INSTANCE_OF] -> (ot:ObjectType {objectType: e2e.objectType})
        MATCH (e1) - [:INSTANCE_OF] -> (et1:EventType)
        MATCH (e2) - [:INSTANCE_OF] -> (et2:EventType)
        MATCH p = (et1) - [:DF* {objectType: e2e.objectType}] -> (et2) // find all ets inbetween e1 ad=nd e2
        WITH o, ot, e1, e2, e2e,  nodes(p) as inbetween_ets
        WHERE size(inbetween_ets) > 0 // check there are more than 0 instances of ets
        UNWIND RANGE(1, size(inbetween_ets) - 2) AS i // unwind and for each et, infer missing event
        WITH o, ot,  e1, e2, i, e2e, inbetween_ets[i] as missing_et
        MATCH (missing_et) - [e2o_type] -> (ot)
        MERGE (new_e:Event {id:"merged"+e1.id, timestamp: e1.timestamp}) - [:INSTANCE_OF] -> (missing_et)
        WITH e2e, new_e, e2o_type, e1, e2, o
        CALL apoc.merge.relationship(new_e, type(e2o_type),{},{},o,{})
        YIELD rel as e2o_rel
        WITH e2e, e1, e2, collect(new_e) as new_events // now we also have to update DFs
        WITH e2e, e1, e2, new_events, new_events[0] as first_new, new_events[-1] AS last_new
        DELETE e2e
        MERGE (e1) - [:DF {objectType: e2e.objectType, id:e2e.id}] -> (first_new) // attach e1 to first new event
        MERGE (last_new) - [:DF {objectType: e2e.objectType, id:e2e.id}] -> (e2) //attach e2 to last new event
        WITH new_events, e2e
        UNWIND RANGE(0, size(new_events) - 2) AS i
        WITH e2e, new_events[i] as first, new_events[i+1] as second
        MERGE (first) - [:DF {objectType: e2e.objectType, id:e2e.id}] -> (second) // add dfs inbetween
RETURN null
'''